## Databases and SQL with Python

**Goal:** Learn to interact with databases (both SQL and NoSQL) using Python.

---

**Module 1: SQL Fundamentals**

**(1.1) Basic SQL Syntax**

SQL (Structured Query Language) is the standard language for interacting with relational databases.  Here's a breakdown of core commands:

*   **`SELECT`**: Retrieves data from one or more tables.
*   **`FROM`**: Specifies the table(s) to retrieve data from.
*   **`WHERE`**: Filters the results based on a condition.
*   **`ORDER BY`**: Sorts the results in ascending (ASC) or descending (DESC) order.
*   **`GROUP BY`**: Groups rows with the same values in specified columns.
*   **`JOIN`**: Combines rows from two or more tables based on a related column.  Common types include `INNER JOIN`, `LEFT JOIN`, `RIGHT JOIN`, and `FULL JOIN`.
*   **`INSERT INTO`**: Adds new rows to a table.
*   **`UPDATE`**: Modifies existing rows in a table.
*   **`DELETE FROM`**: Removes rows from a table.

**Example (Conceptual -  assume a table named `users` with columns `id`, `name`, `age`, `city`):**

```sql
-- Select all columns and rows from the 'users' table
SELECT * FROM users;

-- Select only the 'name' and 'age' columns
SELECT name, age FROM users;

-- Select users older than 25
SELECT * FROM users WHERE age > 25;

-- Select users from 'New York', ordered by name
SELECT * FROM users WHERE city = 'New York' ORDER BY name ASC;

-- Count users in each city
SELECT city, COUNT(*) AS user_count FROM users GROUP BY city;

-- (Assuming a 'orders' table with 'user_id' and 'order_date')
-- Join users and orders to get user names and their order dates
SELECT users.name, orders.order_date
FROM users
INNER JOIN orders ON users.id = orders.user_id;

-- Insert a new user
INSERT INTO users (name, age, city) VALUES ('Alice', 30, 'London');

-- Update Alice's age
UPDATE users SET age = 31 WHERE name = 'Alice';

-- Delete Alice
DELETE FROM users WHERE name = 'Alice';

```

**(1.2) Database Types: Relational (SQL) vs. NoSQL**

*   **Relational Databases (SQL):**
    *   Data is organized into tables with rows (records) and columns (fields).
    *   Relationships between tables are defined using foreign keys.
    *   Examples:  MySQL, PostgreSQL, SQLite, Oracle, SQL Server.
    *   Use SQL for querying and managing data.
    *   ACID properties (Atomicity, Consistency, Isolation, Durability) are often emphasized.

*   **NoSQL Databases:**
    *   "Not Only SQL" - a broad category of databases that don't adhere to the strict relational model.
    *   Various data models:
        *   **Document databases** (e.g., MongoDB): Store data in JSON-like documents.
        *   **Key-value stores** (e.g., Redis):  Store data as key-value pairs.
        *   **Wide-column stores** (e.g., Cassandra):  Store data in flexible columns.
        *   **Graph databases** (e.g., Neo4j):  Store data as nodes and relationships.
    *   Often prioritize scalability and flexibility over strict consistency.
    *   May use different query languages or APIs.

---

**Module 2: Python Database Connectors**

This module introduces the libraries used to connect to different databases from Python.

**(2.1) sqlite3 (for SQLite)**

SQLite is a lightweight, file-based database.  `sqlite3` is built into Python (no extra installation needed!).

```python
import sqlite3

# Connect to a database (or create it if it doesn't exist)
conn = sqlite3.connect('mydatabase.db')

# Create a cursor object
cursor = conn.cursor()

# Create a table
cursor.execute('''
    CREATE TABLE IF NOT EXISTS users (
        id INTEGER PRIMARY KEY,
        name TEXT,
        age INTEGER,
        city TEXT
    )
''')

# Commit the changes and close the connection
conn.commit()
conn.close()

print("SQLite database 'mydatabase.db' and 'users' table created (or connected to).")
```

**(2.2) psycopg2 (for PostgreSQL)**

PostgreSQL is a powerful, open-source relational database.

```python
# Install:  pip install psycopg2-binary  (or psycopg2 for more complex setups)
import psycopg2

try:
    # Connect to your PostgreSQL database (replace with your connection details)
    conn = psycopg2.connect(
        database="mydatabase",
        user="myuser",
        password="mypassword",
        host="localhost",
        port="5432"
    )

    # Create a cursor object
    cursor = conn.cursor()

    # Example: Create a table (if it doesn't exist)
    cursor.execute('''
        CREATE TABLE IF NOT EXISTS products (
            id SERIAL PRIMARY KEY,
            name TEXT,
            price NUMERIC
        )
    ''')

    # Commit the changes
    conn.commit()
    print("Successfully connected to PostgreSQL and created (or connected to) 'products' table.")

except psycopg2.Error as e:
    print(f"Error connecting to PostgreSQL: {e}")

finally:
    if conn:
        cursor.close()
        conn.close()
        print("PostgreSQL connection closed.")

```

**(2.3) pymysql (for MySQL)**

MySQL is another popular, open-source relational database.

```python
# Install: pip install pymysql
import pymysql

try:
    # Connect to your MySQL database (replace with your connection details)
    conn = pymysql.connect(
        host='localhost',
        user='myuser',
        password='mypassword',
        database='mydatabase'
    )

    # Create a cursor
    cursor = conn.cursor()

    # Example: Create a table (if it doesn't exist)
    cursor.execute('''
        CREATE TABLE IF NOT EXISTS customers (
            id INT AUTO_INCREMENT PRIMARY KEY,
            name VARCHAR(255),
            email VARCHAR(255)
        )
    ''')

    # Commit the changes
    conn.commit()
    print("Successfully connected to MySQL and created (or connected to) 'customers' table.")

except pymysql.Error as e:
    print(f"Error connecting to MySQL: {e}")

finally:
    if conn:
        cursor.close()
        conn.close()
        print("MySQL connection closed.")
```

**(2.4) pymongo (for MongoDB - NoSQL)**

MongoDB is a popular document-based NoSQL database.

```python
# Install: pip install pymongo
from pymongo import MongoClient

try:
    # Connect to your MongoDB server (replace with your connection string)
    client = MongoClient('mongodb://localhost:27017/')  # Default MongoDB port

    # Access a database (or create it if it doesn't exist)
    db = client['mydatabase']

    # Access a collection (similar to a table in SQL)
    collection = db['mycollection']

    # Example: Insert a document
    document = {"name": "John Doe", "age": 30, "city": "New York"}
    result = collection.insert_one(document)
    print(f"Inserted document with ID: {result.inserted_id}")

    # Example: Find all documents
    for doc in collection.find():
        print(doc)

    print("Successfully connected to MongoDB and performed operations.")


except Exception as e:
    print(f"Error connecting to MongoDB: {e}")

finally:
    if client:
        client.close()
        print("MongoDB connection closed.")
```

---

**Module 3: Database Operations in Python**

This module demonstrates how to perform CRUD (Create, Read, Update, Delete) operations.  We'll use SQLite for simplicity, but the principles apply to other databases (with slight syntax adjustments).

```python
import sqlite3

def create_user(conn, name, age, city):
    """Inserts a new user into the database."""
    cursor = conn.cursor()
    # Parameterized query (prevents SQL injection!)
    cursor.execute("INSERT INTO users (name, age, city) VALUES (?, ?, ?)", (name, age, city))
    conn.commit()
    print(f"User {name} created.")

def get_users(conn):
    """Retrieves all users from the database."""
    cursor = conn.cursor()
    cursor.execute("SELECT * FROM users")
    rows = cursor.fetchall()  # Fetch all results
    for row in rows:
        print(row)  # Each row is a tuple

def get_user_by_id(conn, user_id):
    """Retrieves a user by their ID."""
    cursor = conn.cursor()
    cursor.execute("SELECT * FROM users WHERE id = ?", (user_id,))
    row = cursor.fetchone()  # Fetch a single result (or None if not found)
    return row

def update_user_city(conn, user_id, new_city):
    """Updates the city of a user."""
    cursor = conn.cursor()
    cursor.execute("UPDATE users SET city = ? WHERE id = ?", (new_city, user_id))
    conn.commit()
    print(f"User ID {user_id} updated.")

def delete_user(conn, user_id):
    """Deletes a user by their ID."""
    cursor = conn.cursor()
    cursor.execute("DELETE FROM users WHERE id = ?", (user_id,))
    conn.commit()
    print(f"User ID {user_id} deleted.")

# --- Main Execution ---
conn = sqlite3.connect('mydatabase.db')

# Create some users
create_user(conn, 'Bob', 28, 'Chicago')
create_user(conn, 'Charlie', 35, 'Miami')

# Get and print all users
print("All users:")
get_users(conn)

# Get a user by ID
user = get_user_by_id(conn, 2) #ID is likely to be 2 or higher, not necessarily consecutive after deletion
if user:
    print(f"User with ID 2: {user}")
else:
    print("User with ID 2 not found.")

# Update a user's city
update_user_city(conn, 1, 'San Francisco')

# Delete a user
delete_user(conn, 2)

print("All users after updates/deletes:")
get_users(conn)

conn.close()
```

**Key Points:**

*   **Parameterized Queries:**  Always use parameterized queries (e.g., `cursor.execute("... (?, ?, ?)", (value1, value2, value3))`) to prevent SQL injection vulnerabilities.  *Never* directly embed user input into SQL strings.
*   **`fetchall()` vs. `fetchone()` vs. `fetchmany()`:**  These methods control how many rows are retrieved from the cursor after executing a query.
*   **Error Handling:**  Real-world applications should include robust error handling (try-except blocks) to gracefully handle database connection issues, query errors, etc.
*   **Transactions:**  For operations that involve multiple steps (e.g., transferring money between accounts), use transactions to ensure that all steps either succeed or fail together (atomicity).

---

**Module 4: (Optional) ORM (Object-Relational Mapper) - Briefly Introduce**

**(4.1) Concept of ORMs (e.g., SQLAlchemy)**

An ORM is a library that maps database tables to Python objects.  This allows you to interact with the database using object-oriented programming instead of writing raw SQL.

**(4.2) Benefits and Drawbacks of Using ORMs**

*   **Benefits:**
    *   **Abstraction:**  You work with objects and methods, hiding the underlying SQL.
    *   **Code Reusability:**  Define your models once and reuse them throughout your application.
    *   **Database Portability:**  Switching between different database systems (e.g., SQLite to PostgreSQL) can be easier.
    *   **Security:** ORMs often provide built-in protection against SQL injection.
*   **Drawbacks:**
    *   **Learning Curve:**  You need to learn the ORM's API.
    *   **Performance Overhead:**  In some cases, ORMs can be slower than carefully optimized raw SQL.
    *   **Complexity:**  For very complex queries, ORMs can become cumbersome.

**Example (SQLAlchemy - very brief):**

```python
# Install: pip install sqlalchemy
from sqlalchemy import create_engine, Column, Integer, String
from sqlalchemy.orm import declarative_base, sessionmaker

# Define the database connection
engine = create_engine('sqlite:///mydatabase.db', echo=True)  # echo=True shows generated SQL
Base = declarative_base()

# Define a model (representing a table)
class User(Base):
    __tablename__ = 'users'
    id = Column(Integer, primary_key=True)
    name = Column(String)
    age = Column(Integer)
    city = Column(String)

# Create the table (if it doesn't exist)
Base.metadata.create_all(engine)

# Create a session
Session = sessionmaker(bind=engine)
session = Session()

# Create a new user object
new_user = User(name='David', age=40, city='Seattle')

# Add the user to the session and commit
session.add(new_user)
session.commit()

# Query for users
for user in session.query(User).filter(User.age > 30):
    print(user.name, user.age)

session.close()

```

This example just scratches the surface of SQLAlchemy.  It's a powerful and flexible ORM, but it has a steeper learning curve than using the basic database connectors directly.  This ORM section is OPTIONAL and you can skip it for now if you are just beginning.

This mini-course provides a solid foundation for working with databases in Python. Remember to practice and adapt the examples to your specific needs and database systems. Good luck!
